In [ ]:
from dendron import *
from dendron.hf.actions.image_lm_action import ImageLMActionConfig, ImageLMAction

from PIL import Image
import requests

In [ ]:
cfg = ImageLMActionConfig(load_in_4bit=True,
                          max_new_tokens=70,
                          do_sample=True,
                          top_p=0.95,
                          device="cuda",
                          use_flash_attn_2=True,
                          model_name="llava-hf/llava-1.5-13b-hf")

In [ ]:
node = ImageLMAction("ilm_action", cfg)
tree = BehaviorTree(node)

In [ ]:
url = "https://upload.wikimedia.org/wikipedia/commons/a/af/Cara_de_quem_caiu_do_caminh%C3%A3o..._%28cropped%29.jpg"
img = Image.open(requests.get(url, stream=True).raw)
img

In [ ]:
prompt_template = "USER: <image>\n{query}\nASSISTANT:"

In [ ]:
import re

def trunc(node):
    out_str = node.blackboard[node.output_key]
    m = re.search(r'ASSISTANT:(.*)', out_str, re.DOTALL)
    node.blackboard[node.output_key] = m.group(1).strip()

tree.root.set_post_tick(trunc)

In [ ]:
tree.blackboard_set("text_in", prompt_template.format(query="What kind of puppy is this? Is this a basset hound? Give a short answer."))
tree.blackboard_set("image_in", img)

In [ ]:
status = tree.tick_once()

In [ ]:
if status == NodeStatus.SUCCESS:
    print(tree.blackboard["out"])

In [ ]:
tree.pretty_print()